In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding

OpenAIEmbedding()

In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import json

with open('E-9_Visa_Docs.json', 'r') as f:
    visa_docs_json = json.load(f)

""" Load TextNodes from JSON file """

from llama_index.core.schema import TextNode

visa_nodes = [
    TextNode(
        text=doc["text"],
        metadata=doc["metadata"],
        excluded_llm_metadata_keys=doc["excluded_llm_metadata_keys"],
        excluded_embed_metadata_keys=doc["excluded_embed_metadata_keys"],
    )
    for doc in visa_docs_json
]

In [5]:
import faiss

# dimensions of text-ada-embedding-002
d = 1536
faiss_index = faiss.IndexFlatL2(d)

INFO:faiss.loader:Loading faiss.
Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
Successfully loaded faiss.


In [10]:
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    visa_nodes, storage_context=storage_context
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [11]:
# save index to disk
index.storage_context.persist(persist_dir="faiss")

In [ ]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./faiss")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./faiss"
)
index = load_index_from_storage(storage_context=storage_context)